In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Step 1: 加载数据
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
# train=DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True)
# test=DataLoader(test_dataset,batch_size=len(test_dataset),shuffle=False)
# Step 2: 读取小批量样本
batch_size = 64

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Step 3: 定义模型
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Step 4: 初始化模型参数
model = MLPModel()

# Step 5: 定义损失函数
criterion = nn.CrossEntropyLoss()

In [4]:
# Step 6: 训练模型
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 10

train_losses = []  # 保存每个epoch的总体训练误差
test_losses = []   # 保存每个epoch的总体测试误差
train_accuracies = []  # 保存每个epoch的训练准确率
test_accuracies = []   # 保存每个epoch的测试准确率

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    total_train_samples = 0
    correct_train = 0
    for batch in train_loader:
        data, target = batch
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * data.size(0)
        pred_train = output.argmax(dim=1, keepdim=True)
        correct_train += pred_train.eq(target.view_as(pred_train)).sum().item()
        total_train_samples += data.size(0)

    average_train_loss = total_train_loss / total_train_samples
    train_losses.append(average_train_loss)
    train_accuracy = correct_train / total_train_samples
    train_accuracies.append(train_accuracy)

    # 在每个epoch结束后计算测试误差
    model.eval()
    total_test_loss = 0
    correct_test = 0
    total_test_samples = 0
    with torch.no_grad():
        for batch in test_loader:
            data, target = batch
            output = model(data)
            total_test_loss += criterion(output, target).item() * data.size(0)
            pred_test = output.argmax(dim=1, keepdim=True)
            correct_test += pred_test.eq(target.view_as(pred_test)).sum().item()
            total_test_samples += data.size(0)

    average_test_loss = total_test_loss / total_test_samples
    test_losses.append(average_test_loss)
    test_accuracy = correct_test / total_test_samples
    test_accuracies.append(test_accuracy)

    # 输出每个epoch的总体测试误差和准确率
    # print(f'Epoch {epoch+1}/{num_epochs}, Average Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# 输出所有epoch结束后的模型的总体训练误差、测试误差和准确率
print(f'Final Training Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}')
print(f'Final Test Loss: {test_losses[-1]:.4f}, Test Accuracy: {test_accuracies[-1]:.4f}')


Final Training Loss: 0.2022, Train Accuracy: 0.9427
Final Test Loss: 0.1990, Test Accuracy: 0.9435
